In [228]:
import datetime 
import pandas as pd
import requests
from bs4 import BeautifulSoup
# import pymongo
import warnings
from dateutil.parser import parse
import datetime as dt
from lxml import etree
import random
import re
import numpy as np
from datetime import date
from datetime import datetime, timedelta

warnings.filterwarnings("ignore")
# client = pymongo.MongoClient('127.0.0.1', 27017)
# db = client["news"]
# newsCollection = db["news_collection"]

proxies = {
    "https" : "fnyproxy.fnylocal.com:8080",
    "http" : "fnyproxy.fnylocal.com:8080"
}

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0'}




url = "https://tradingeconomics.com/calendar"


res = requests.get(url, proxies = proxies,headers=headers, verify = False)
soup = BeautifulSoup(res.text, "html.parser")



In [229]:
data = pd.read_html(res.content)[1]


dates = data.columns[0]

df = data
df.columns = df.columns.get_level_values(0)
column_names = df.columns.values
column_names[0] = 'time'
column_names[1] = 'country'
column_names[2] = 'indicator'
column_names[7] = 'remove1'
column_names[8] = 'remove2'
df.columns = column_names


df = df.drop(columns=['remove1', 'remove2'])

dates = list(dates)


In [230]:
df['date'] = None

# 'time' sütunundaki son PM'den AM'ye geçtiğinde, dates listesinden yeni bir tarih alınacak
current_date_index = 0
current_am_pm = None

for i in range(len(df)):
    time_value = df.loc[i, 'time']
    # time_value'yi string olarak ele al ve boş veya NaN değerleri kontrol et
    if pd.notna(time_value):
        time_value = str(time_value)
        if 'AM' in time_value or 'PM' in time_value:
            am_pm = 'AM' if 'AM' in time_value else 'PM'
            if current_am_pm is None:
                current_am_pm = am_pm  # İlk değeri atama
            elif current_am_pm == 'PM' and am_pm == 'AM':
                current_date_index += 1  # Yeni tarih bilgisine geçiş

            df.loc[i, 'date'] = dates[current_date_index]
            current_am_pm = am_pm
    else:
        df.loc[i, 'time'] = 'Unknown'  # veya df.drop(i) ile satırı sil
        df.loc[i, 'date'] = dates[current_date_index]

# print(df)

df.to_excel("calendar_data.xlsx") 

In [231]:
today = date.today()
yesterday = today - timedelta(1)

d2 = today.strftime("%B %d %Y")



today_df =df[df['date'].str.contains(d2)]
today_df = today_df[today_df['Actual'].notna()]
today_df = today_df[today_df['Forecast'].notna()]

# d1 = yesterday.strftime("%B %d %Y")

# yesterday_df =df[df['date'].str.contains(d1)]
# yesterday_df = yesterday_df[yesterday_df['Actual'].notna()]
# yesterday_df = yesterday_df[yesterday_df['Forecast'].notna()]


#test için
# turkce_sehir = {'CN': 'Çin', 'KR': 'Kore', 'US': 'ABD', 'JP': 'Japonya', 'ZA': 'Güney Afrika', 'ES': 'İspanya'}
turkce_sehir = {'US': 'ABD'}
country_list = list(turkce_sehir.keys())


filtered_df = today_df[today_df['country'].isin(country_list)]

In [232]:
turkce_keys_df = pd.DataFrame(columns = ["ay", "indicator", "aciklanan", "beklenti", "sehir", "fark", "tarih"])





turkce_ay = {'JAN': 'Ocak', 'FEB': 'Şubat',  'JUN': 'Haziran', 'MAY': 'Mayıs', 'JUL': 'Temmuz', 'AUG': 'Ağustos', 'NOV': 'Kasım', 'DEC': 'Aralık', 'Q1': 'birinci çeyerk', 'Q2': 'ikinci çeyrek', 'Q3': 'üçüncü çeyrek', 'Q4': 'dördüncü çeyrek'}
months = list(turkce_ay.keys())

# test için
# turkce_indikator= {'Employment Change': 'İstihdam Değişimi', 'Part Time Employment': 'Yarı-zamanlı istihdam', 'HMRC Payrolls Change': 'HMRC Bordro Değişikliği',
#                    'Average Earnings': 'Ortalama Kazanç', 'Retail Sales MoM': 'Perakende Satış Ayı', 'Public Sector Net Borrowing Ex': 'Kamu Kesimi Net Borçlanma Örneği',
#                   'Loan Prime Rate 1Y': 'Kredi Prime Oranı 1 Yıl', 'Loan Prime Rate 5Y': 'Kredi Prime Oranı 5 Yıl', '5-Year KTB Auction': '5 Yıllık KTB Müzayedesi',
#                   'Chicago Fed National Activity Index': 'Chicago Fed Ulusal Faaliyet Endeksi', 'Core Inflation Rate YoY': ' Yıllık Asıl Enflansyon Oranı', 
#                   'Jibun Bank Manufacturing PMI Flash': ['Jubun Bankası Servis Flash', 'Jubun Bankası Servisini Flash'],
#                   'Jibun Bank Services PMI Flash': ['Jubun Bankası Servis', 'Jubun Bankası Servisini'], 'Core Inflation Rate MoM': [' Aylık Asıl Enflansyon Oranı', 'Aylık Asıl Enflansyon Oranı'],
#                   'Retail Sales MoM': ['Japon retail satışları', 'Japon retail satışlarının'], 'Inflation Rate MoM Prel': ['ABD ISM imalat PMI','İspanyol aylık enflasyon oranının'],
#                   'Inflation Rate YoY Prel': ['ABD yıllık manşet TÜFE','İspanya yıllık enf oranının'],
#                   'Core Inflation Rate YoY Prel': ['JOLTs açık iş pozisyonu'],
#                   'Harmonised Inflation Rate MoM Prel': ['ABD yıllık çekirdek TÜFE'],
#                   'Harmonised Inflation Rate YoY Prel': ['Tarım dışı istihdam'],
                   
                   
                   
#                   'ISM Manufacturing PMI': ['ABD ISM imalat PMI', "ABD ISM imalat PMI'ının"], 'JOLTs Job Openings': ['JOLTs açık iş pozisyonu', "JOLTs açık iş pozisyonu verisinin"],
#                   'Non Farm Payrolls': ['Tarım dışı istihdam', "Tarım dışı istihdam verisinin"], 'ISM Services PMI': ['ABD ISM hizmet PMI', "ABD ISM hizmet PMI'ının"], 
#                   'Core Inflation Rate MoM': [' ABD aylık çekirdek TÜFE', " ABD aylık çekirdek TÜFE'nin"], 'Core Inflation Rate YoY': ['ABD yıllık çekirdek TÜFE', "ABD yıllık çekirdek TÜFE'nin"], 
#                   'Inflation Rate MoM': ['ABD aylık manşet TÜFE', "ABD aylık manşet TÜFE'nin"], 'Inflation Rate YoY': ['ABD yıllık manşet TÜFE', "ABD yıllık manşet TÜFE'nin"], 
#                   'Core PCE Price Index MoM': ['Çekirdek PCE', "Çekirdek PCE'nin"], 
#                   'PCE Price Index MoM': ['Manşet PCE', "Manşet PCE'nin"], 'Fed Interest Rate Decision': ["Fed'in politika faizi kararı", "Fed'in politika faizi kararının"]}




turkce_indikator= {'ISM Manufacturing PMI': ['ABD ISM imalat PMI', "ABD ISM imalat PMI'ının"], 'JOLTs Job Openings': ['JOLTs açık iş pozisyonu', "JOLTs açık iş pozisyonu verisinin"],
                  'Non Farm Payrolls': ['Tarım dışı istihdam', "Tarım dışı istihdam verisinin"], 'ISM Services PMI': ['ABD ISM hizmet PMI', "ABD ISM hizmet PMI'ının"], 
                  'Core Inflation Rate MoM': [' ABD aylık çekirdek TÜFE', " ABD aylık çekirdek TÜFE'nin"], 'Core Inflation Rate YoY': ['ABD yıllık çekirdek TÜFE', "ABD yıllık çekirdek TÜFE'nin"], 
                  'Inflation Rate MoM': ['ABD aylık manşet TÜFE', "ABD aylık manşet TÜFE'nin"], 'Inflation Rate YoY': ['ABD yıllık manşet TÜFE', "ABD yıllık manşet TÜFE'nin"], 
                  'Core PCE Price Index MoM': ['Çekirdek PCE', "Çekirdek PCE'nin"], 
                  'PCE Price Index MoM': ['Manşet PCE', "Manşet PCE'nin"], 'Fed Interest Rate Decision': ["Fed'in politika faizi kararı", "Fed'in politika faizi kararının"]}

desired_indicators = list(turkce_indikator.keys())




desired_filtered_df = filtered_df[filtered_df["indicator"].str.contains('|'.join(desired_indicators))]

def turkce_deger(value):
    if '%' in value:
        value = value.replace('%', '')
        value = "%.2f" % round(float(value), 2)
        value = '%' + str(value)

    elif isinstance(value, float) or isinstance(value, int):
        value = "%.2f" % round(float(value), 2)
    
        
    if 'T' in value:
        value = value.replace('T', '')
        value =   value + ' trilyon'

    if 'B' in value:
        value = value.replace('B', '')
        value =   value + ' milyar'

    if 'M' in value:
        value = value.replace('M', '')
        value =   value + ' milyon'

    if 'K' in value:
        value = value.replace('K', '')
        value =   value + ' bin'

        
    if '.' in value:
        value = value.replace('.', ',')
 
        
        
    if '£' in value:
        value = value.replace('£', '')
        value =   value + ' pound'

    if '$' in value:
        value = value.replace('$', '')
        value =   value + ' dolar'

        
    return value




count_lower = random.randint(0, 2)
count_higher = random.randint(0, 2)
count_not_significant = random.randint(0, 2)


In [233]:
def turkce_yapici(desired_row):
    global turkce_keys_df
    # print('turkce girdi')
    
    indicator_name = desired_row['indicator']
    month = indicator_name.rsplit(' ', 1)[1]
    month = month.split('/', 1)[0]
    
    indicator_name = indicator_name.rsplit(' ', 1)[0]
    if month in months:
        ay = turkce_ay[month]
    else:
        indicator_name = indicator_name + ' ' + month
        ay = ''
        ay_var = False
    
    try:
        indicator_name = turkce_indikator[indicator_name]
        indicator_name = indicator_name[0]
        
    except:
        indicator_name = indicator_name
        print('indikator adı türkçeye çevrilmemiş !!!!!!!!!!!!!!!!!!!')
    
    sehir = desired_row['country']
    try:
        sehir = turkce_sehir[sehir]
    except:
        sehir = sehir
        print('sehir adının türkçesi girilmemeiş !!!!!!!!!!!!!!!!!!!')
        
    
    actual = desired_row['Actual']
    actual = turkce_deger(actual)
    forecast = desired_row['Forecast']
    forecast = turkce_deger(forecast)
    fark = float(re.sub('[^\d\.-]', '', desired_filtered_df.iloc[index_row]['Actual'])) - float(re.sub('[^\d\.-]', '', desired_filtered_df.iloc[index_row]['Forecast']))
    tarih = desired_row['date']
    new_row = {"ay": ay, "indicator": indicator_name, "aciklanan": actual, "beklenti": forecast, "sehir": sehir, 'fark': fark, 'tarih': tarih}

    turkce_keys_df = turkce_keys_df.append(new_row, ignore_index=True)
    
 
#     print(row_df['country'])
#     print(row_df['indicator'])
#     print(row_df['Actual'])
#     print(row_df['Forecast'])

In [234]:
for index_row in range(0,len(desired_filtered_df)):
    

    turkce_yapici(desired_filtered_df.iloc[index_row])
    


turkce_keys_df.to_excel("turkce_keys.xlsx", index = False, engine = 'openpyxl')



In [235]:
keys = pd.read_excel('~/Ozgur/calendar/turkce_keys.xlsx')
keys

try:
    manset = keys[keys['indicator'].str.contains('manşet')]
    

    if len(manset) == 2:
        yillik_manset = manset[manset['indicator'].str.contains('yıllık')]
        aylik_manset = manset[manset['indicator'].str.contains('aylık')]
        keys = keys.drop(manset.index)

    
        #manset için
        if (yillik_manset['fark'] == 0).bool() and (aylik_manset['fark'] != 0).bool():
            manset_secilen = aylik_manset
            keys = keys.append(manset_secilen)

        else:
            manset_secilen = yillik_manset 
            keys = keys.append(manset_secilen)
    
except:
    print('nasıl')
    
  
    
try:
    cekirdek = keys[keys['indicator'].str.contains('çekirdek')]
    
    
    
    if len(cekirdek) == 2:
        yillik_cekirdek = cekirdek[cekirdek['indicator'].str.contains('yıllık')]
        aylik_cekirdek = cekirdek[cekirdek['indicator'].str.contains('aylık')]
        keys = keys.drop(cekirdek.index)
    
    
        #cekirdek için
        if (yillik_cekirdek['fark'] == 0).bool() and (aylik_cekirdek['fark'] != 0).bool():
            cekirdek_secilen = aylik_cekirdek
            keys = keys.append(cekirdek_secilen)
        else:
            cekirdek_secilen = yillik_cekirdek
            keys = keys.append(cekirdek_secilen)
except:
    print('ney')
    
    

df = keys


def create_sentence(row):
    
    #ortak
    verisi = ["rakamları", "verileri", "rakamı", "verisi"]
    düzeyindeki = ["seviyesindeki", "düzeyindeki"]
    ##
    
    
    beklentilerin = ["öngörülerin", "beklentilerin"]
    
    ## ilk cümle
    seviyesinde = ["düzeyinde", "seviyesinde", "ile"]
    
    altında_kaldı = ["aşağısında kaldı", "gerisinde kaldı", "altında kaldı", "aşağısında gerçekleşti"]
    ustunde_açıklandı = ["yukarısında açıklandı", "üstünde açıklandı", "üstünde gerçekleşti", "yukarısında gerçekleşti"]
    ##
    
    
    #### ikinci cümle
    altında = ["aşağısında", "gerisinde", "altında"]
    ustunde = ["üstünde", "yukarısında"]
    seviyesinde_gerçelkeşti = ["olarak açıklandı", "seviyesinde gerçekleşti", "düzeyinde gerçekleşti"]
    ##

    
    
    
    verisi = np.random.choice(verisi)
    seviyesinde = np.random.choice(seviyesinde)
    düzeyindeki = np.random.choice(düzeyindeki)
    beklentilerin = np.random.choice(beklentilerin)
    
    ## ikinci cümle
    # altında = np.random.choice(altında)
    seviyesinde_gerçelkeşti = np.random.choice(seviyesinde_gerçelkeşti)
    ##
    aciklanan = (row['aciklanan'])
    beklenti = (row['beklenti'])
    indicator = row['indicator']
    

    
    if row['fark'] < 0:        # beklenti altı
        category = 'under'
        altust = np.random.choice(altında_kaldı)
        altust2 = np.random.choice(altında)
    elif row['fark'] > 0:
        category = 'over'
        altust = np.random.choice(ustunde_açıklandı)
        altust2 = np.random.choice(ustunde)
    else:
        category = 'equal'
        beklentilere = ['beklentilere']
        beklentilerin = np.random.choice(beklentilere)
        altust = 'paralel geldi'
        altust2 = 'paralel'
        


    templates = [f'{indicator} {verisi} {aciklanan} {seviyesinde} gerçekleşerek {beklenti} {düzeyindeki} {beklentilerin} {altust}.',
            f'{indicator} {verisi} {beklenti} {düzeyindeki} {beklentilerin} {altust2} kalarak {aciklanan} {seviyesinde_gerçelkeşti}.']

    print(np.random.choice(templates))



for index in range(0,len(df)):
    create_sentence(df.iloc[index])


In [237]:
data

,time,country,indicator,Actual,Previous,Consensus,Forecast,remove1,remove2
0,01:30 AM,AU,RBA Chart Pack,NaN,NaN,NaN,NaN,NaN,NaN
1,03:00 AM,CN,Balance of Trade JUL,$84.65B,$99.05B,$99B,$90B,NaN,NaN
2,03:00 AM,CN,Exports YoY JUL,7%,8.6%,9.7%,7.8%,NaN,NaN
3,03:00 AM,CN,Imports YoY JUL,7.2%,-2.3%,3.5%,2.5%,NaN,NaN
4,03:00 AM,ID,Foreign Exchange Reserves JUL,$145.4B,$140.2B,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
404,05:00 PM,US,Baker Hughes Total Rigs Count AUG/16,NaN,NaN,NaN,NaN,NaN,NaN
405,05:25 PM,US,Fed Goolsbee Speech,NaN,NaN,NaN,NaN,NaN,NaN
406,NaN,KR,50-Year KTB Auction,NaN,3.035%,NaN,NaN,NaN,NaN
407,NaN,ZA,SACCI Business Confidence JUN,NaN,107.8,NaN,113,NaN,NaN
